In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 한글 폰트
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# 그래프 마이너스 폰트 깨지는 문제 대처
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import mglearn

# 원핫인코딩(가변수)
- 범주형 변수를 표현하는 데 가장 널리 쓰이는 방법 (one-hot-encoding)
- 범주형 변수를 0 또는 1값을 가진 하나 이상의 새로운 특성으로 바꾼 것
- 통계학의 '더미 코딩'과 비슷하지만, 여기서는 k-1개의 특성으로 변환하는 것이 아님(모두가 0인 특성은 없음)

# pandas의 get_dummies() 이용한 범주형 변수 변환
- 객체 타입(문자열 같은), 범주형 타입을 가진 열을 자동으로 변환해줌
- 숫자로된 속성은 모두 연속형이라 생각함 -> 문자열 속성으로 변경해주거나 컬럼 명시해줘야함!

In [4]:
# 1. pandas로 처리
import os
data = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'), header=None, index_col=False,
                  names = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship',
                           'race','gender','capital-gain','capital-loss','hours-per-week','native-country', 'income'])
# 예제용으로 컬럼 몇개만 선택
data = data [['age','workclass','education','gender','hours-per-week','occupation','income']].copy()
data.head()

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


In [6]:
data.gender.value_counts()

 Male      21790
 Female    10771
Name: gender, dtype: int64

In [7]:
print('원본 특성:\n', list(data.columns), '\n')
data_dummies = pd.get_dummies(data)
print('get_dummies 후의 특성:\n', list(data_dummies.columns))

원본 특성:
 ['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income'] 

get_dummies 후의 특성:
 ['age', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'gender_ Female', 'gender_ Male', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct', 'occupation_ 

- 연속형 특성인 age와 hours-per-week은 그대로이며, 범주형 특성은 값마다 새로운 특성으로 확장됨

In [10]:
data_dummies.head()

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,39,40,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,50,13,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,38,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,53,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,28,40,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [12]:
# data_dummies의 values 속성을 이용해 DataFrame을 NumPy배열로 바꿀 수 있음 -> 이를 이용해 머신러닝 모델 학습 시킴
# 타깃값인 income 속성따로 분리해야됨

# 타깃값 제외한 특성
features = data_dummies.loc[:, 'age':'occupation_ Transport-moving']    # pandas 열 indexing은 범위 끝을 포함함!!!

# NumPy 배열 추출
X = features.values
y = data_dummies['income_ >50K'].values     # 원래는 income 속성은 타깃값이라, 원핫인코딩 안하고 그대로 사용해도됨
print(f'X.shape {X.shape}, y.shape: {y.shape}')

X.shape (32561, 44), y.shape: (32561,)


In [15]:
# 로지스틱 회귀 (분류용) 적용
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('테스트 점수: {:.2f}'.format(logreg.score(X_test, y_test)))

테스트 점수: 0.81


C:\Users\qkrwl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 숫자로 표현된 범주형 특성
- 숫자로 표현됐지만, 값 사이의 어떤 순서도 없을 때 범주형/이산적이라고 생각해야함
- pd.get_dummies()함수는 숫자 특성은 모두 연속형이라 생각함
    - 따라서, 문자형 특성으로 변경해주거나, 변경할 열 명시해줘야함

In [16]:
# test : 숫자 특성과 범주형 문자열 특성을 가진 df 생성
demo_df = pd.DataFrame({'숫자 특성':[0,1,2,1], '범주형 특성':['양말','여우','양말','상자']})
demo_df

,숫자 특성,범주형 특성
0,0,양말
1,1,여우
2,2,양말
3,1,상자


In [17]:
pd.get_dummies(demo_df)   # 문자열 특성만 인코딩되며, 숫자특성은 변환되지 않음

,숫자 특성,범주형 특성_상자,범주형 특성_양말,범주형 특성_여우
0,0,0,1,0
1,1,0,0,1
2,2,0,1,0
3,1,1,0,0


### 문자열 특성으로 변환 후, 원핫인코딩 적용

In [18]:
demo_df['숫자 특성'] = demo_df['숫자 특성'].astype(str)
pd.get_dummies(demo_df)

,숫자 특성_0,숫자 특성_1,숫자 특성_2,범주형 특성_상자,범주형 특성_양말,범주형 특성_여우
0,1,0,0,0,1,0
1,0,1,0,0,0,1
2,0,0,1,0,1,0
3,0,1,0,1,0,0


### 열 지정해서 원핫 인코딩 적용

In [19]:
pd.get_dummies(demo_df, columns = ['숫자 특성','범주형 특성'])

,숫자 특성_0,숫자 특성_1,숫자 특성_2,범주형 특성_상자,범주형 특성_양말,범주형 특성_여우
0,1,0,0,0,1,0
1,0,1,0,0,0,1
2,0,0,1,0,1,0
3,0,1,0,1,0,0


# scikit-learn의 OneHotEncoder로 범주형 변수 변환
- 훈련 세트와 테스트 세트를 같은 방식으로 다루기 쉬운 것이 장점
- 모든 열에 인코딩을 수행하기 때문에 주의 필요!
- sparse=False로 설정하면 OneHotEncoder가 희소 행렬이 아닌, NumPy 배열을 반환함!

In [21]:
from sklearn.preprocessing import OneHotEncoder
# sparse=False로 설정하면 OneHotEncoder가 희소 행렬이 아닌, NumPy 배열을 반환함!
ohe = OneHotEncoder(sparse=False)
print(ohe.fit_transform(demo_df))

[[1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]


- 문자열 특성과 정수 특성 모두 변환됨
- scikit-learn의 출력은 df가 아니기때문에 열 이름이 없음 -> 원본 범주형 이름을 얻으려면 get_feature_names메서드 사용

In [24]:
print(ohe.get_feature_names())

['x0_0' 'x0_1' 'x0_2' 'x1_상자' 'x1_양말' 'x1_여우']


# scikit-learn의 ColumnTransformer()로 범주형, 연속형 변수 각기 다른 변환 적용
- 입력 데이터의 열마다 다른 변환 적용할 수 있음
    - 연속형 특성과 범주형 특성은 다른 종류의 전처리 과정이 필요하기 때문에 매우 유용
- **객체 생성 시 지정: [이름, 변환기 객체(변환 방법), 변환이 적용될 열], ...**
- fit, transform 메서드 사용 가능
- 단점
    - sklearn 0.20 ver. 기준으로 변환된 출력 열에 대응하는 입력 열을 찾지 못함
    - 그저 순서대로 변환하니까, 순서대로 대응해볼 뿐, 원형 이름 찾는 메서드 없음

## ColumnTransformer

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# ColumnTransformer 객체 생성
ct = ColumnTransformer([
        ('scaling', StandardScaler(), ['age','hours-per-week']),
        ('onehot',OneHotEncoder(sparse=False), ['workclass','education','gender','occupation'])
            ])

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# income을 제외한 모든 열 추출
data_features = data.drop('income', axis=1)

# 데이터프레임과 income을 train, test로 분할
X_train, X_test, y_train, y_test = train_test_split(data_features, data.income, random_state=0)

ct.fit(X_train)
X_train_trans = ct.transform(X_train)   # train set 변환
X_train_trans.shape

(24420, 44)

- 연속형 특성은 스케일 조정, 범주형 특성은 원핫인코딩 됨

In [32]:
logreg = LogisticRegression()
logreg.fit(X_train_trans, y_train)

X_test_trans = ct.transform(X_test)    # test set 변환
print('테스트 점수: {:.2f}'.format(logreg.score(X_test_trans, y_test)))

테스트 점수: 0.81


C:\Users\qkrwl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
# ColumnTransformer 안의 단계 접근 - named_transformers_ 속성 사용
ct.named_transformers_.onehot    # 객체 생성시 지정했던 이름으로 조회 'onehot','scaling'

OneHotEncoder(sparse=False)

In [34]:
ct.named_transformers_.scaling

StandardScaler()

## make_column_transformer
- ColumnTransformer에서 변환 이름을 일일이 지정해줘야 했음
- 클래스 이름을 기반으로 자동으로 각 단계에 이름을 붙여주는 함수
- [변환 적용할 열이름, 변환 방법],...

In [36]:
from sklearn.compose import make_column_transformer

ct = make_column_transformer(
            (['age','hours-per-week'], StandardScaler()),
            (['workclass','education','gender','occupation'], OneHotEncoder(sparse=False))
                )